### additinoal_analysis_combination_eval_annotators
- 最終更新: 2024/2/14, 17:10

- 追加分析: combinationモデルの評価（研究2のベストモデルの評価）

#### Google Driveのマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 環境構築

aptやらpipやらの下準備
- 参考: https://qiita.com/karaage0703/items/30485c2ba1c396760982

In [2]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install fugashi ipadic unidic-lite
!pip install scikit-learn

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
Suggested packages:
  apt-xapian-index aptitude-doc-en | aptitude-doc debtags tasksel libcwidget-dev xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
0 upgraded, 5 newly installed, 0 to remove and 33 not upgraded.
Need to get 3,838 kB of archives.
After this operation, 17.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aptitude-common all 0.8.13-3ubuntu1 [1,719 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsigc++-2.0-0v5 amd64 2.10.4-2ubuntu3 [12.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcwidget4 amd64 0.5.18-5build1 [306 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libxapian30 amd64 1

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [4]:
#transformersの準備
!pip install transformers[torch] neologdn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 15.9 MB/s eta 0:00:00
  Created wheel for neologdn: filename=neologdn-0.5.2-cp310-cp310-linux_x86_64.whl size=219144 sha256=3fdf41d84ed86328da122057ed807422cb2fa31dd0813b6fecf45f18169d690c
  Stored in directory: /root/.cache/pip/wheels/bd/87/af/2a36d74f666a8428943b70d71c5e9dd740435bf671f210672c
Successfully built neologdn


In [5]:
pip install accelerate -U

In [6]:
#GPUの認識状況の確認
!nvidia-smi

Wed Feb 14 08:53:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#### 以下はコード

In [7]:
#ライブラリのインポート
import numpy as np
import pandas as pd
import tqdm

In [ ]:
import torch
from torch import nn
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from transformers import AdamW

In [ ]:
#乱数のseedを全固定する
#ref: https://qiita.com/ku_a_i/items/bac3befedbadab45774f
import random
import os
RANDOM_STATE = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(RANDOM_STATE)

##### モデルの読み込み

In [ ]:
#Study 2のベストモデルのディレクトリ
bert_model_dir = "/content/drive/MyDrive/Colaboratory/Work/sst_auto_scoring/model/study2/study2_answer_and_rubric_train_and_eval/checkpoint-580"
bert_model = BertForSequenceClassification.from_pretrained(bert_model_dir)

##### データの読み込み
- 筆頭著者、第二著者、アノテータ1、アノテータ2

In [ ]:
#全員の分類結果を読みこむ
data_annotator = pd.read_excel('/content/drive/MyDrive/Colaboratory/Work/sst_auto_scoring/data/additional_analysis/20240214_pick_and_classifying_samples_22_percents_four_annotators.xlsx')
data_annotator = data_annotator.query('PickedSample == 1')
data_annotator_target = data_annotator[["order", "word_pair", "answer", "point_oka", "point_ksm", "point_hosoi", "point_mizukoshi"]]

#word_pairをバラしたり、reset_indexしたり
data_annotator_target = pd.concat([data_annotator_target, data_annotator_target['word_pair'].str.split('-', expand=True)], axis=1).drop('word_pair', axis=1)
data_annotator_target = data_annotator_target.rename(columns={0: 'word1', 1: 'word2'})
data_annotator_target = data_annotator_target.reindex(columns=["order", "word1", "word2", "answer", "point_oka", "point_ksm", "point_hosoi", "point_mizukoshi"])
data_annotator_target = data_annotator_target.reset_index().drop('index', axis=1)
data_annotator_target

,order,word1,word2,answer,point_oka,point_ksm,point_hosoi,point_mizukoshi
0,3,鳥,飛行機,飛ぶ,2.0,2.0,2.0,2.0
1,4,時間,川,流れる,2.0,2.0,2.0,2.0
2,16,思考,網,張り巡らす,2.0,2.0,2.0,2.0
3,17,美術館,墓,建物,0.0,0.0,0.0,0.0
4,28,酒場,教会,お酒,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
435,2262,山,障害物,出っ張っているもの,0.0,1.0,1.0,1.0
436,2271,美術館,墓,過去のものが収められている,2.0,2.0,2.0,2.0
437,2285,山,障害物,そびえ立つイメージ,2.0,0.0,1.0,1.0
438,2293,思考,網,何かを捕えようとしている所,1.0,1.0,0.0,1.0


In [ ]:
#なぜかword1/word2にスペースが混ざっているケースがあったので削除する
def remove_dumb_spaces(string):
  return string.rstrip().lstrip()

#データセットのための準備
data_annotator_target["word1"] = data_annotator_target["word1"].map(remove_dumb_spaces)
data_annotator_target["word2"] = data_annotator_target["word2"].map(remove_dumb_spaces)
data_annotator_target["text"] = data_annotator_target['word1'] + "[SEP]" + data_annotator_target["word2"] + "[SEP]" + data_annotator_target["answer"]
data_annotator_target

,order,word1,word2,answer,point_oka,point_ksm,point_hosoi,point_mizukoshi,text
0,3,鳥,飛行機,飛ぶ,2.0,2.0,2.0,2.0,鳥[SEP]飛行機[SEP]飛ぶ
1,4,時間,川,流れる,2.0,2.0,2.0,2.0,時間[SEP]川[SEP]流れる
2,16,思考,網,張り巡らす,2.0,2.0,2.0,2.0,思考[SEP]網[SEP]張り巡らす
3,17,美術館,墓,建物,0.0,0.0,0.0,0.0,美術館[SEP]墓[SEP]建物
4,28,酒場,教会,お酒,0.0,0.0,0.0,0.0,酒場[SEP]教会[SEP]お酒
...,...,...,...,...,...,...,...,...,...
435,2262,山,障害物,出っ張っているもの,0.0,1.0,1.0,1.0,山[SEP]障害物[SEP]出っ張っているもの
436,2271,美術館,墓,過去のものが収められている,2.0,2.0,2.0,2.0,美術館[SEP]墓[SEP]過去のものが収められている
437,2285,山,障害物,そびえ立つイメージ,2.0,0.0,1.0,1.0,山[SEP]障害物[SEP]そびえ立つイメージ
438,2293,思考,網,何かを捕えようとしている所,1.0,1.0,0.0,1.0,思考[SEP]網[SEP]何かを捕えようとしている所


In [ ]:
#tokenizerの準備
MODEL_NAME = "cl-tohoku/bert-base-japanese"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

#datasetの準備
from datasets import Dataset
def create_dataset(data):
  """
  dataset（pandasのデータフレーム）をtorchで使えるデータセットにする
  """

  def tokenize_basic(batch):
      return tokenizer(batch["text"], padding=True, truncation=True)

  dataset = Dataset.from_pandas(data)
  dataset_encoded = dataset.map(tokenize_basic, batched=True, batch_size=None)

  return dataset_encoded

dataset = create_dataset(data_annotator_target)
dataset

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


Map:   0%|          | 0/440 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dataset({
    features: ['order', 'word1', 'word2', 'answer', 'point_oka', 'point_ksm', 'point_hosoi', 'point_mizukoshi', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 440
})

### `bert_model`を使って、分類を試みる


In [ ]:
#cudaを使う準備
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [ ]:
#modelをtrainerで読み込み
#ref: https://discuss.huggingface.co/t/saving-loading-model-in-colab-and-making-predictions/6723/5
from transformers import TrainingArguments
from transformers import Trainer
from transformers import EarlyStoppingCallback

trainer = Trainer(model=bert_model)
trainer.model = bert_model.cuda()

In [ ]:
#モデルの出力を手に入れる
preds_output = trainer.predict(dataset)
preds_output.predictions.shape

(440, 3)

In [ ]:
#0,1,2のどのラベルのoddsが高いかで出力
preds_output_label = np.argmax(preds_output.predictions, axis=1)
preds_output_label

array([2, 2, 2, 1, 1, 1, 0, 0, 0, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 0, 1, 1,
       2, 1, 2, 2, 2, 1, 2, 0, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 0, 1, 0, 2,
       2, 2, 1, 2, 0, 1, 0, 1, 1, 2, 1, 0, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 0, 1, 1, 1, 2, 1,
       1, 2, 1, 1, 1, 1, 1, 2, 2, 0, 1, 2, 1, 1, 2, 0, 2, 2, 2, 2, 1, 1,
       2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2,
       2, 2, 1, 2, 0, 0, 2, 1, 1, 2, 1, 0, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2,
       1, 2, 1, 2, 1, 1, 0, 1, 2, 1, 2, 1, 0, 2, 2, 2, 1, 2, 2, 1, 0, 0,
       1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 0, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 0, 1, 1, 1,
       0, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 2,
       1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 0, 1, 2, 1, 1,
       2, 2, 2, 1, 1, 2, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 2, 2, 1, 2, 1,
       2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2,

In [ ]:
#modelの出力をdataframeに結合
data_annotator_target["pred"] = preds_output_label
data_annotator_target

,order,word1,word2,answer,point_oka,point_ksm,point_hosoi,point_mizukoshi,text,pred
0,3,鳥,飛行機,飛ぶ,2.0,2.0,2.0,2.0,鳥[SEP]飛行機[SEP]飛ぶ,2
1,4,時間,川,流れる,2.0,2.0,2.0,2.0,時間[SEP]川[SEP]流れる,2
2,16,思考,網,張り巡らす,2.0,2.0,2.0,2.0,思考[SEP]網[SEP]張り巡らす,2
3,17,美術館,墓,建物,0.0,0.0,0.0,0.0,美術館[SEP]墓[SEP]建物,1
4,28,酒場,教会,お酒,0.0,0.0,0.0,0.0,酒場[SEP]教会[SEP]お酒,1
...,...,...,...,...,...,...,...,...,...,...
435,2262,山,障害物,出っ張っているもの,0.0,1.0,1.0,1.0,山[SEP]障害物[SEP]出っ張っているもの,2
436,2271,美術館,墓,過去のものが収められている,2.0,2.0,2.0,2.0,美術館[SEP]墓[SEP]過去のものが収められている,2
437,2285,山,障害物,そびえ立つイメージ,2.0,0.0,1.0,1.0,山[SEP]障害物[SEP]そびえ立つイメージ,1
438,2293,思考,網,何かを捕えようとしている所,1.0,1.0,0.0,1.0,思考[SEP]網[SEP]何かを捕えようとしている所,1


In [ ]:
#predictionが結合されたデータセットの吐き出し
data_annotator_target.to_csv("/content/drive/MyDrive/Colaboratory/Work/sst_auto_scoring/result/additional_analysis/data_annotator_target_with_prediction.csv", index=False)

##### predictionのデータを読み込む

In [8]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.metrics import classification_report

In [9]:
data_annotator_target_dir = "/content/drive/MyDrive/Colaboratory/Work/sst_auto_scoring/result/additional_analysis/data_annotator_target_with_prediction.csv"
data_annotator_target = pd.read_csv(data_annotator_target_dir)
data_annotator_target

,order,word1,word2,answer,point_oka,point_ksm,point_hosoi,point_mizukoshi,text,pred
0,3,鳥,飛行機,飛ぶ,2.0,2.0,2.0,2.0,鳥[SEP]飛行機[SEP]飛ぶ,2
1,4,時間,川,流れる,2.0,2.0,2.0,2.0,時間[SEP]川[SEP]流れる,2
2,16,思考,網,張り巡らす,2.0,2.0,2.0,2.0,思考[SEP]網[SEP]張り巡らす,2
3,17,美術館,墓,建物,0.0,0.0,0.0,0.0,美術館[SEP]墓[SEP]建物,1
4,28,酒場,教会,お酒,0.0,0.0,0.0,0.0,酒場[SEP]教会[SEP]お酒,1
...,...,...,...,...,...,...,...,...,...,...
435,2262,山,障害物,出っ張っているもの,0.0,1.0,1.0,1.0,山[SEP]障害物[SEP]出っ張っているもの,2
436,2271,美術館,墓,過去のものが収められている,2.0,2.0,2.0,2.0,美術館[SEP]墓[SEP]過去のものが収められている,2
437,2285,山,障害物,そびえ立つイメージ,2.0,0.0,1.0,1.0,山[SEP]障害物[SEP]そびえ立つイメージ,1
438,2293,思考,網,何かを捕えようとしている所,1.0,1.0,0.0,1.0,思考[SEP]網[SEP]何かを捕えようとしている所,1
